# Animal Classifier Training Pipeline

This notebook trains a deep learning model to classify images of animals (birds, cats, and dogs) using transfer learning with MobileNetV2.

**Key Features:**
- Transfer learning with MobileNetV2 pretrained on ImageNet
- Experiment tracking with Weights & Biases (WandB)
- Automated train/validation split
- Model export to SavedModel format

**Training Configuration:**
- Image size: 150x150
- Batch size: 16
- Epochs: 10
- Optimizer: Adam (lr=1e-3)
- Classes: birds, cats, dogs

## 1. Environment Setup

First, we'll verify GPU availability and set up our experiment tracking.

### Check GPU Availability

In [ ]:
import tensorflow as tf

# Check if GPU is available
gpus = tf.config.list_physical_devices("GPU")
print(f"GPUs available: {len(gpus)}")
gpus

### Initialize Weights & Biases

WandB will track our experiments, log metrics, and save model checkpoints.

In [ ]:
import wandb

# Login to WandB (you'll be prompted for your API key)
wandb.login()

## 2. Data Preparation

We'll upload and extract the dataset containing images of birds, cats, and dogs.

### Upload Dataset

Upload your `images.zip` file when prompted.

In [ ]:
from google.colab import files

# Upload images.zip
uploaded = files.upload()

### Extract Dataset

Unzip the uploaded file to access the images.

In [ ]:
!unzip -q images.zip -d .
print("✓ Dataset extracted successfully")

### Create Train/Validation Split

We'll split the dataset into training (80%) and validation (20%) sets for each class.

**Directory Structure:**
```
images/
├── train/
│   ├── birds/
│   ├── cats/
│   └── dogs/
└── val/
    ├── birds/
    ├── cats/
    └── dogs/
```

In [ ]:
import os
import random
import shutil
from pathlib import Path

# Set random seed for reproducibility
random.seed(42)

# Configuration
DATA_DIR = Path("images")
TRAIN_RATIO = 0.8
classes = ["dogs", "cats", "birds"]

# Create train and validation directories for each class
for split in ["train", "val"]:
    for c in classes:
        (DATA_DIR / split / c).mkdir(parents=True, exist_ok=True)

# Split images for each class
for c in classes:
    class_dir = DATA_DIR / c
    images = list(class_dir.glob("*"))
    random.shuffle(images)

    # Calculate split index
    split_idx = int(len(images) * TRAIN_RATIO)
    train_imgs = images[:split_idx]
    val_imgs = images[split_idx:]

    # Copy images to respective directories
    for p in train_imgs:
        shutil.copy(p, DATA_DIR / "train" / c / p.name)
    for p in val_imgs:
        shutil.copy(p, DATA_DIR / "val" / c / p.name)

    print(f"{c}: {len(train_imgs)} training, {len(val_imgs)} validation")

print("\n✓ Data split completed successfully")

## 3. Model Training

We'll train a classifier using transfer learning with MobileNetV2 as the base model.

### Install Dependencies and Import Libraries

In [ ]:
!pip install -q wandb tensorflow

import tensorflow as tf
import wandb
from wandb.integration.keras import (
    WandbMetricsLogger,
    WandbModelCheckpoint,
)

print(f"TensorFlow version: {tf.__version__}")
print(f"WandB version: {wandb.__version__}")

### Initialize WandB Experiment

Configure the experiment with hyperparameters that will be tracked.

In [ ]:
# Initialize WandB run
wandb.init(
    project="animal-classifier",
    config={
        "img_size": 150,
        "batch_size": 16,
        "epochs": 10,
        "lr": 1e-3,
    }
)

# Access the config
config = wandb.config
print(f"Experiment configuration: {dict(config)}")

### Load and Prepare Datasets

Create TensorFlow datasets from the directory structure with automatic label encoding.

In [ ]:
IMG_SIZE = (config.img_size, config.img_size)

# Load training dataset
train_ds = tf.keras.utils.image_dataset_from_directory(
    "images/train",
    image_size=IMG_SIZE,
    batch_size=config.batch_size,
    label_mode="categorical"
)

# Load validation dataset
val_ds = tf.keras.utils.image_dataset_from_directory(
    "images/val",
    image_size=IMG_SIZE,
    batch_size=config.batch_size,
    label_mode="categorical"
)

# Log class names to WandB
print(f"Class order: {train_ds.class_names}")
wandb.config.update({"classes": train_ds.class_names})

### Build Model Architecture

**Architecture:**
1. **Rescaling Layer**: Normalize pixel values from [0, 255] to [0, 1]
2. **MobileNetV2 Base**: Pretrained convolutional base (frozen)
3. **Global Average Pooling**: Reduce spatial dimensions
4. **Dense Output Layer**: 3 neurons with softmax activation for classification

In [ ]:
# Normalization layer
norm = tf.keras.layers.Rescaling(1./255)

# Load pretrained MobileNetV2 base
base = tf.keras.applications.MobileNetV2(
    input_shape=IMG_SIZE + (3,),
    include_top=False,
    weights="imagenet"
)
base.trainable = False  # Freeze base model weights

# Build complete model
model = tf.keras.Sequential([
    norm,
    base,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(3, activation="softmax")
])

print("\nModel Summary:")
model.summary()

### Compile Model

Configure the model with optimizer, loss function, and metrics.

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(config.lr),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

print("✓ Model compiled successfully")

### Train Model

Train the model with WandB callbacks for:
- Metrics logging (loss, accuracy)
- Model checkpointing (saves best model)

In [ ]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=config.epochs,
    callbacks=[
        WandbMetricsLogger(),
        WandbModelCheckpoint("best_model.keras", save_best_only=True)
    ]
)

print("\n✓ Training completed successfully")

## 4. Model Export and Download

Export the trained model and prepare it for deployment.

### Export to SavedModel Format

Export the model in TensorFlow's SavedModel format for deployment.

In [ ]:
# Export model to SavedModel format
export_path = "models/pets/1"
model.export(export_path)

print(f"\n✓ Model exported to: {export_path}")

### Create ZIP Archive

Package the saved model for easy download and deployment.

In [ ]:
!zip -r pets_savedmodel.zip models/pets

print("\n✓ Model archive created: pets_savedmodel.zip")

### Download Model

Download the zipped model to your local machine.

In [ ]:
from google.colab import files

files.download("pets_savedmodel.zip")

print("✓ Model download initiated")

## Summary

**Training Results:**
- View your training metrics and experiments on [Weights & Biases](https://wandb.ai)
- The best model is saved as `best_model.keras`
- The exported model is available in `pets_savedmodel.zip`

**Next Steps:**
1. Deploy the SavedModel using TensorFlow Serving
2. Use the model for inference in production
3. Fine-tune the model if needed by unfreezing base layers